In [26]:
import math
import numpy as np

In [165]:
class GraphNet:
    # Make an empty graph on n nodes with linear activation
    def __init__(self, n):
        self.size = n
        self.adj = np.zeros((n, n))
        self.weights = np.zeros((n, n))
        self.act = 'linear'
        self.input_nodes = []
        self.output_nodes = []
        self.state = [0 for i in range(self.size)]
        self.mode = 'overwrite'
        self.time = 0
    
    # Sets the mode for the step method
    # Options: overwrite, add
    def mode(self, m):
        self.mode = m
    
    # Connect an edge from node i to node j
    def connect(self, i, j, w=1):
        self.adj[j,i] = w
    
    # Change activation on node i
    # Options: linear, relu, sigmoid/logistic, tanh
    def activation(self, i, f):
        self.act = f
    
    # Get the list of edge weights that are inputs to node i
    def inputs(self, i):
        return self.adj[i,:]
    
    # Get the list of edge weights that are outputs from node i
    def outputs(self, i):
        return self.adj[:,i]
    
    # Reset weights
    def reset_weights(self):
        self.weights = np.copy(self.adj)
    
    # Initialize random edge weights, assuming edge weights are all 0 or 1
    def init_random(self):
        self.weights = np.random.rand(self.size, self.size)*self.adj
    
    # Normalize columns to satisfy Markov state transition rules
    def init_markov(self):
        self.init_random()
        for i in range(self.size):
            colsum = sum(self.weights[:,i])
            if colsum != 0:
                for j in range(self.size):
                    self.weights[j,i] /= colsum
    
    # Make the vector of indices be the input nodes
    def define_input(self, v):
        self.input_nodes = v
    
    # Make the vector of indices be the output nodes
    def define_output(self, v):
        self.output_nodes = v
    
    # Reset state
    def reset_state(self):
        self.state = [0 for i in range(self.size)]
    
    # Runs one time step of the network, with the given input values, and returns the current output values
    def step(self, input_vals):
        self.time += 1
        if self.mode == 'overwrite':
            for i in range(len(self.input_nodes)):
                self.state[self.input_nodes[i]] = input_vals[i]
        if self.mode == 'add':
            for i in range(len(self.input_nodes)):
                self.state[self.input_nodes[i]] += input_vals[i]
        self.state = np.dot(self.weights, self.state)
        output_vals = [self.state[self.output_nodes[i]] for i in range(len(self.output_nodes))]
        return output_vals
    
    # Runs several time steps of the network, with the given vector of input value vectors at each time step, 
    # and returns a vector of output values
    def steps(self, input_vals_v, n=0):
        output_vals_v = []
        empty_input = [0 for i in range(len(self.input_nodes))]
        for t in range(max(n, len(input_vals_v))):
            input_vals = empty_input
            if t < len(input_vals_v):
                input_vals = input_vals_v[t]
            output_vals_v.append(self.step(input_vals))
        return output_vals_v
    
    # Run n steps on a input only at initial step
    def stepn(self, input_vals_0, n=1):
        output_vals_v = []
        empty_input = [0 for i in range(len(self.input_nodes))]
        for t in range(n):
            input_vals = empty_input
            if t == 0:
                input_vals = input_vals_0
            output_vals_v.append(self.step(input_vals))
        return output_vals_v

In [166]:
test = GraphNet(4)
test.connect(0,1)
test.connect(1,2)
test.connect(2,1)
test.connect(2,3)
test.define_input([0])
test.define_output([3])

In [167]:
test.adj
test.init_markov()
test.weights

array([[0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.55796508, 0.        ],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.44203492, 0.        ]])

In [168]:
test.stepn([1], 10)

[[0.0],
 [0.0],
 [0.44203492419103935],
 [0.0],
 [0.2466400499864614],
 [0.0],
 [0.13761653418822178],
 [0.0],
 [0.07678521993089758],
 [0.0]]

In [172]:
test.reset_state()
test.steps([[1],[0.5],[0.25],[0.125]], 10)

[[0.0],
 [0.0],
 [0.44203492419103935],
 [0.22101746209551967],
 [0.35714878103422126],
 [0.17857439051711063],
 [0.19927654668483713],
 [0.09963827334241857],
 [0.11118935347795303],
 [0.055594676738976515]]